In [126]:
!pip3 install glove-python
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import operator
from sklearn.model_selection import train_test_split
from glove import Corpus, Glove
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding

In [127]:
# read the dataset 
data = pd.read_csv("dataset.csv")
print(data.shape)
print(data.head())

(10560, 12)
    hmid  ...                     duration
0  27674  ...                   half_a_day
1  27685  ...                   half_a_day
2  27691  ...            at_least_one_hour
3  27701  ...  all_day_im_still_feeling_it
4  27712  ...                 a_few_moment

[5 rows x 12 columns]


In [128]:
# Get Moment, agency and social
target_data = data[['moment', 'agency', 'social']]
print(target_data)

                                                  moment agency social
0      I was happy when my son got 90% marks in his e...     no    yes
1              went to movies with my friends it was fun    yes    yes
2      A hot kiss with my girl friend last night made...    yes    yes
3      My son woke me up to a fantastic breakfast of ...     no    yes
4      My older daughter keeps patting my younger dau...     no    yes
5          I cooked my girlfriend a wonderful breakfast.    yes    yes
6         My Mother gave me a surprise visit at my home.     no    yes
7      There was hardly any traffic on my way to work...    yes     no
8                     I came to my office at right time.    yes     no
9      The day I got my degree in industrial engineering    yes     no
10     I went to office hour of one of my professors,...    yes    yes
11     We all ladies member from my family went for a...    yes    yes
12     When my wife came home from work and we shared...    yes    yes
13    

In [129]:
# Data Analysis
global agency
global social
agency = list(target_data['agency'])
moments = list(target_data['moment'])
social = list(target_data['social'])
yy = 0
yn = 0
nn = 0
ny = 0
# print("Len: ",len(moments))
for i in range(len(moments)):
    if agency[i] == 'yes' and social[i] =='yes':
        yy += 1
    elif agency[i] == 'yes' and social[i] == 'no':
        yn += 1
    elif agency[i] == 'no' and social[i] =='yes':
        ny += 1
    else:
        nn += 1

print("------------------ Social -------------------")
print("               YES        NO      SUM")
print("Agency YES    ",yy,"     ",yn,"  ",yy+yn)
print("       NO     ",ny,"     ",nn,"   ",ny+nn)
print("       SUM    ",yy+ny,"     ",yn+nn,"  ",yy+nn+ny+yn)


------------------ Social -------------------
               YES        NO      SUM
Agency YES     3554       4242    7796
       NO      2071       693     2764
       SUM     5625       4935    10560


In [0]:
# Observation: Data is positive in high proportion although there is an imbalance as more positive data
# for social than for agency. This might result in poor accuracy while predicting the label agency.

In [0]:
from nltk.tokenize import RegexpTokenizer
import statistics
def tokenizer(sentence):
    tokenizer = RegexpTokenizer('[a-zA-Z0-9\']+')
    words = tokenizer.tokenize(sentence)
    return words

In [0]:
from nltk.corpus import stopwords
def stopWordRemoval(words):
    filtered_data = []
    stop_words = set(stopwords.words('english'))
    for word in words:
        if not word in stop_words:
            if word.isnumeric() or word.isalpha():
                filtered_data.append(word)
    return filtered_data
    

In [132]:
# Data Pre-processing
global sentence_list
global sequenced_vocab
global sentences
global vocab
# 1. Split the sentences in moments into words (using regex tokenizer)
vocab = list()
sequenced_vocab = dict()
index = 1
sentence_list = list()
seq_list = list()
sentences = list()

for i in range(len(moments)):
    moments[i] = moments[i].lower()
    bag_of_words = tokenizer(moments[i])
    #remove stop words
#     bag_of_words = stopWordRemoval(bag_of_words)
    sentence_list.append(bag_of_words)
    vocab += bag_of_words

# sentences = sentence_list

# Form sequence dictionary and convert each sentence into a number sequence 
for word in vocab:
    if word not in sequenced_vocab.keys():
        sequenced_vocab[word] = index
        index += 1
print("Unique words in Vocabulary : ",len(sequenced_vocab))
# print("Sequenced Vocab : ")
# print(sequenced_vocab)

# Avg length of each sentence
sentence_length = [ len(sentence_list[i]) for i in range(len(sentence_list))]
sentence_length.sort()
print("------------------ SENTENCE STATISTICS OF MOMENTS -----------------")
print("Minimum Length : ",sentence_length[0])
print("Maximum Length : ",sentence_length[len(sentence_length)-1])
print("Average Length : ",round(sum(sentence_length)/len(sentence_length)))
print("Median Length  : ", round(statistics.median(sentence_length)))
# print(sentence_list)

Unique words in Vocabulary :  7604
------------------ SENTENCE STATISTICS OF MOMENTS -----------------
Minimum Length :  2
Maximum Length :  70
Average Length :  13
Median Length  :  12


In [134]:
# 2. Assign a sequence of numbers to each of the sentences 
global sequenced_vocab
for i in range(len(sentence_list)):
    for word in sentence_list[i]:
        if word in sequenced_vocab.keys():
            seq_list.append(sequenced_vocab[word])
    sentence_list[i] = seq_list
    seq_list = []
    
# print(" SEQUENCED SENTENCES  ")
# print(sentence_list)

#Printing Sentence Stats
count_len = dict()
for length in sentence_length:
    if length not in count_len.keys():
        count_len[length] = 1
    else:
        count_len[length] += 1

print("Total Sentences : ",len(sentence_list))
print("Length of majority sentences: ",max(count_len.items(), key=operator.itemgetter(1))[0])
    
# word_list = list(sequenced_vocab.keys())
# indexes = list(sequenced_vocab.values())
# print("Indexs: ",indexes)

# # 3. Pad with 0s if sentence length is not same (at the beginning)
padded_sentence = pad_sequences(sentence_list, maxlen=20)
print(" ================= PADDED SENTENCE =================")
print(padded_sentence[500])

Total Sentences :  10560
Length of majority sentences:  8
 ================= PADDED SENTENCE =================
[   0    0    0    0    0    0    0    0    0    0    0    0    0    1
  363 1431   16   20  616   23]


In [0]:
#transform agency and social to binary labels
global agency_label
global social_label
agency_label = [ 0 if agency[i] == "no" else 1 for i in range(len(agency))]
social_label = [ 0 if social[i] == "no" else 1 for i in range(len(social))]
# print(agency_label)
# print(social_label)

In [0]:
def createEmbMatrix(vectors, word_dict):
  max_words = len(list(word_dict.keys()))
  embedding_matrix = np.zeros((max_words, 100))
  for i in range(max_words):
    embedding_vector = vectors[i]
    embedding_matrix[i] = embedding_vector
  return embedding_matrix

In [0]:
def runCNNModel(sentences_train, y_train, sentences_test, y_test):
  model = Sequential()
  model.add(Embedding(7920,100,input_length=20))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv1D(64,3,padding='valid',activation='relu',strides=1))
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(256))
  # model.add(layers.Dropout(0.2))
  model.add(layers.Activation('relu'))
  model.add(layers.Dense(1))
  model.add(layers.Activation('sigmoid'))
  model.summary()
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  history = model.fit(np.array(sentences_train), np.array(y_train),
                      epochs=10,
                      batch_size=32,
                      verbose=1,
                      validation_data=(np.array(sentences_test), np.array(y_test)))

In [138]:
#Using Pre-Trained Embedding GLoVe for the Embedding Layer
# corpus = Corpus()
# # print(sentences)
# corpus.fit(padded_sentence.tolist(), window=5)
# glove = Glove(no_components=100, learning_rate=0.05)
# glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=True)
# # Add the object to the dictionary
# glove.save('glove.model')
# glove.add_dictionary(corpus.dictionary)
# print(glove.word_vectors.shape)
# print("VOCAB SIZE: ",len(list(glove.dictionary.keys())))
# vocab_size = len(list(glove.dictionary.keys()))

# ----------------------------------------------- Without GloVe Embedding Matrix -------------------------------------------
# Split the data into train and validate
sentences_train,sentences_test,y_train,y_test = train_test_split(
                                                padded_sentence.tolist(), social_label,  
                                                test_size=0.25,  
                                                random_state=1000)
print("----------- TRAIN ---------------")
print("Shape: ",len(sentences_train))
print("----------- VALIDATE --------------")
print("Shape: ",len(sentences_test))

# Glove Embedding Matrix
embedding_matrix = createEmbMatrix(glove.word_vectors, glove.dictionary)
print(embedding_matrix.shape)

# CNN for Social and Agency
print("================================ For SOCIAL ==============================")
runCNNModel(sentences_train, y_train, sentences_test, y_test)

print("============================= For AGENCY =================================")
sentences_train,sentences_test,y_train,y_test = train_test_split(
                                                padded_sentence.tolist(), agency_label,  
                                                test_size=0.25,  
                                                random_state=1000)
print("----------- TRAIN ---------------")
print("Shape: ",len(sentences_train))
print("----------- VALIDATE --------------")
print("Shape: ",len(sentences_test))
runCNNModel(sentences_train, y_train, sentences_test, y_test)

----------- TRAIN ---------------
Shape:  7920
----------- VALIDATE --------------
Shape:  2640
(7347, 100)
================================ For SOCIAL ==============================
Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 20, 100)           792000    
_________________________________________________________________
dropout_38 (Dropout)         (None, 20, 100)           0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 18, 64)            19264     
_________________________________________________________________
global_max_pooling1d_30 (Glo (None, 64)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 256)               16640     
_________________________________________________________________
ac